In [2]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)


from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [3]:
# -*- coding: utf-8 -*-
%matplotlib inline

import tensorflow as tf
from matplotlib import pyplot as plt

shape = (10, 10)
initial_board = tf.random_uniform(shape, minval=0, maxval=2, dtype=tf.int32)

#==================================
import numpy as np
from scipy.signal import convolve2d

def update_board(X):
    # Check out the details at: https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/
    # Compute number of neighbours,
    N = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    # Apply rules of the game
    X = (N == 3) | (X & (N == 2))
    return X
    
board = tf.placeholder(tf.int32, shape=shape, name='board')
board_update = tf.py_func(update_board, [board], [tf.int32])


import matplotlib.animation as animation


#with tf.Session() as session:
session= tf.Session()
initial_board_values = session.run(initial_board)
x_value = session.run(board_update, feed_dict={board: initial_board_values})[0]

fig = plt.figure()
imgplot = plt.imshow(x_value, cmap='Greys',  interpolation='nearest')

X=x_value
def game_of_life(*args):
        global X
        X = session.run(board_update, feed_dict={board: X})[0]
        imgplot.set_array(X)
        return imgplot,

ani = animation.FuncAnimation(fig, game_of_life, interval=1000, blit=True)
    
# plt.show()
display_animation(ani)
#session.close()
# Hint: you will need to remove the plt.show() from the earlier code to make this run!

/usr/local/lib/python2.7/dist-packages/matplotlib/animation.py:782: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)
